In [2]:
# basic imports for connecting to coinbase pro and being able to execute trades
import pandas as pd
import cbpro
from auth_credentials import (api_secret, api_key, api_pass)

# print('test'); making sure everything imported correctly

test


WebsocketClient Class

In [3]:
class TextWebsocketClient(cbpro.WebsocketClient):
    def on_open(self):
        self.url = 'wss://ws-feed-public.sandbox.exchange.coinbase.com'
        self.message_count = 0

    def on_message(self,msg):
        self.message_count += 1
        msg_type = msg.get('type',None)
        if msg_type == 'ticker':
            time_val = msg.get('time',('-'*27))
            price_val = msg.get('price',None)
            price_val = float(price_val) if price_val is not None else 'None'
            product_id = msg.get('product_id',None)

            print(f"{time_val:30} {price_val: .3f} {product_id}\tchannel type:{msg_type}")

    def on_close(self):
        print(f"<---Websocket connection closed--->\n\tTotal messages: {self.message_count}")

In [8]:
# stream = TextWebsocketClient(products=['BTC-USD','ETH-USD'],channels=['ticker'])
# stream.start()

# Trying to test out data stream; not entirely working as expected

---------------------------     16491.180 BTC-USD	channel type:ticker
---------------------------     0.000 ETH-USD	channel type:ticker
Connection is already closed. - data: None
<---Websocket connection closed--->
	Total messages: 2
Connection is already closed. - data: None
<---Websocket connection closed--->
	Total messages: 2
Connection is already closed. - data: None
<---Websocket connection closed--->
	Total messages: 3
Connection is already closed. - data: None
<---Websocket connection closed--->
	Total messages: 3


Client Auth

In [11]:
url = 'https://api-public.sandbox.exchange.coinbase.com'
client = cbpro.AuthenticatedClient(
    api_key,
    api_secret,
    api_pass,
    api_url=url
)

Placing Market Orders


In [12]:
# testing with market order for now; will likely want to place limits and stops
# parameters: size => crypto amount, funds => usd
client.place_market_order(product_id='BTC-USD',side='buy',funds=50)
"""
returns/prints a dict as follows:
{'id': '53ecd854-4d01-431b-939a-bb6dfe6107bd',
 'product_id': 'BTC-USD',
 'side': 'buy',
 'stp': 'dc',
 'funds': '49.70178926',
 'specified_funds': '50',
 'type': 'market',
 'post_only': False,
 'created_at': '2022-12-06T19:03:42.985687Z',
 'fill_fees': '0',
 'filled_size': '0',
 'executed_value': '0',
 'status': 'pending',
 'settled': False}
"""

{'id': '53ecd854-4d01-431b-939a-bb6dfe6107bd',
 'product_id': 'BTC-USD',
 'side': 'buy',
 'stp': 'dc',
 'funds': '49.70178926',
 'specified_funds': '50',
 'type': 'market',
 'post_only': False,
 'created_at': '2022-12-06T19:03:42.985687Z',
 'fill_fees': '0',
 'filled_size': '0',
 'executed_value': '0',
 'status': 'pending',
 'settled': False}

Account History


In [18]:
accounts = client.get_accounts()

for acc in accounts:
    currency = acc.get('currency')
    if currency == 'BTC':
        acc_id = acc.get('id')
        # print(acc_id)

3f664e68-3ab2-47ad-a291-37a02e69b9ee


In [21]:
btc_history = client.get_account_history(acc_id)

In [22]:
import json

# shows trade history in the following format:
"""
{
 "id": "421436199",
 "amount": "0.0050185700000000",
 "balance": "0.0050185700000000",
 "created_at": "2022-12-06T19:03:43.019582Z",
 "type": "match",
 "details": {
  "order_id": "53ecd854-4d01-431b-939a-bb6dfe6107bd",
  "product_id": "BTC-USD",
  "trade_id": "39717525"
 }
"""
# amount is the order amount; balance is the resulting balance in the account
for hist in btc_history:
    print(json.dumps(hist,indent=1))

{
 "id": "421436199",
 "amount": "0.0050185700000000",
 "balance": "0.0050185700000000",
 "created_at": "2022-12-06T19:03:43.019582Z",
 "type": "match",
 "details": {
  "order_id": "53ecd854-4d01-431b-939a-bb6dfe6107bd",
  "product_id": "BTC-USD",
  "trade_id": "39717525"
 }
}
